In [28]:
# なむパイのバージョンを上げる
! pip install numpy==1.16.1

In [ ]:
from keras.models import Sequential
from keras.datasets import imdb
from keras.layers import Dense, Activation, Embedding, LSTM, GRU, SimpleRNN
from keras.optimizers import Adadelta
from keras.utils import np_utils, to_categorical
from keras.preprocessing import sequence
import numpy as np

from matplotlib import pyplot as plt

In [ ]:
# よく使う単語上位20000を定義
max_features = 20000

# 映画の評価が書いてあるデータこれを使用してポジティブネガティブ診断を行う
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = max_features)

max_len  = 180
X_train = sequence.pad_sequences(X_train, maxlen = max_len)
X_test = sequence.pad_sequences(X_test, maxlen = max_len)

In [31]:
# 単語に対するインデックスが入っている、元の文章にふくげんする必要がある
print;(X_train[0])

# 復元
word_index = imdb.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

for k in range(10):
#   上位2万件に入らなかった単語は?に変えられる、行列に入れるため長さを統一するために180単語に合わせるために?で補っている
# LSTMの特性で頭の方の情報は忘れやすいため頭の方に?を付けるようにする
    decoded_review = ' '.join([reverse_word_index.get(i - 3, '?') for i in X_train[k]])
    if y_train[k] == 1:
        print('positive')
    else:
        print('negative')
    print(decoded_review)

positive
father came from the same scottish island as myself so i loved the fact there was a real connection with this film the witty remarks throughout the film were great it was just brilliant so much that i bought the film as soon as it was released for retail and would recommend it to everyone to watch and the fly fishing was amazing really cried at the end it was so sad and you know what they say if you cry at a film it must have been good and this definitely was also congratulations to the two little boy's that played the ? of norman and paul they were just brilliant children are often left out of the praising list i think because the stars that play them all grown up are such a big profile for the whole film but these children are amazing and should be praised for what they have done don't you think the whole story was so lovely because it was true and was someone's life after all that was shared with us all
negative
giant safety pin these are the words to best describe this ter

In [ ]:
# 正解クラスをone-hotエンコーディングして出力
y_train = to_categorical(y_train, 2)
y_test = to_categorical(y_test, 2)

In [ ]:
model = Sequential()

# Embeddingで単語に適当なベクトルを与えている
model.add(Embedding(input_dim = max_features,
                    output_dim = 128,
                    input_length = max_len))

# 最後の入力だけが欲しいので return_sequences = False
model.add(LSTM(32, return_sequences = False))
# Encoder-Decoderのため２つ用意する
model.add(Dense(2))
model.add(Activation('softmax'))
model.compile(loss = 'categorical_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])
model.fit(X_train, y_train, batch_size = 256, epochs = 5, validation_split = 0.1)

Train on 22500 samples, validate on 2500 samples
Epoch 1/5
22500/22500 [==============================] - 24s 1ms/step - loss: 0.4994 - acc: 0.7488 - val_loss: 0.3274 - val_acc: 0.8684
Epoch 2/5
12800/22500 [================>.............] - ETA: 9s - loss: 0.2318 - acc: 0.9114

In [ ]:
# positive
# test_sentence = "It is a wonderful movie that I want to go to see again"

# negative
test_sentence = "This movie is really terrible"

def split_text(text):
    result = []
    for word in text.split():
        word = word.lower()  
    
        try:
            index = word_index[word]
        except(KeyError):
            index = 0
        result.append(index)
      
    return result

x = [split_text(test_sentence)]
# シーケンスを同じ長さになるように詰めます．
x = sequence.pad_sequences(x, maxlen=max_len)
print(x)

# 入力サンプルに対する予測値の出力を生成します．
y = model.predict(x)
print(y)